## EDS 296: Final Mini-Project

Authors: Oksana Protsukha, Leilanie Rubenstein, & Naomi Moraes

This project is assigned as the final assignment for EDS 296 - taught by Dr. Sam Stevenson. In this project we will use geospatial and temporal analysis to investigate changing environmental factors in the western region of India.

### Western Coast of India

We have decided to focus the scope of our investigation into this region - as rising heatwaves (across India) are responsible for increasing numbers of public health impacts - like heat exhaustian, sunstroke, and dehydration. Coupled with the relief/exacerbation cycles of monsoon season - we decided to look at how air surface temperature and precipitation may impact this location over time.

[Maharashtra](images/westindia.png)

In [2]:
# Import packages
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import intake

In [3]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

# Convert the catalog to a df for easier access
cat_df = catalog.df

### Data I/O

We have chosen to use the CanESM5 and CESM2 models - due to abundance of data. As aforementioned we will be using 'tas' and 'pr'. We have also decided to use 'historical' and 'ssp245' experiment_ids, as well as 'r11i1p1f1' member_id - as these variables were found for the desired models and atmospheric variables.

In [55]:
# Specify search terms to query catalog

activity_ids = ['ScenarioMIP', 'CMIP'] 
source_id = ['CanESM5', 'CESM2']
table_id = 'Amon' 
variable_id = ['tas', 'pr'] 
member_id = ['r11i1p1f1']
experiment_id = ['historical', 'ssp245']



In [56]:
res = catalog.search(activity_id=activity_ids, 
                     source_id=source_id,
                     table_id=table_id, 
                     variable_id=variable_id,
                    member_id = member_id,
                    experiment_id = experiment_id)

# Display data frame associated with results
display(res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
1,CMIP,CCCma,CanESM5,historical,r11i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
2,ScenarioMIP,CCCma,CanESM5,ssp245,r11i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
3,ScenarioMIP,CCCma,CanESM5,ssp245,r11i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
4,CMIP,NCAR,CESM2,historical,r11i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190514
5,CMIP,NCAR,CESM2,historical,r11i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190514
6,ScenarioMIP,NCAR,CESM2,ssp245,r11i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/NCAR/CESM2/ss...,NaN,20200528
7,ScenarioMIP,NCAR,CESM2,ssp245,r11i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/NCAR/CESM2/ss...,NaN,20200528


In [63]:
# store historical data for CanESM5
CAN_hist_pr = xr.open_zarr(res.df['zstore'][0], storage_options={'anon': True})
CAN_hist_tas = xr.open_zarr(res.df['zstore'][1], storage_options={'anon': True})

# Store 245 scenario for CanESM5
CAN_245_pr = xr.open_zarr(res.df['zstore'][2], storage_options={'anon': True})
CAN_245_tas = xr.open_zarr(res.df['zstore'][3], storage_options={'anon': True})

# store historical data for CESM2
CESM_hist_pr = xr.open_zarr(res.df['zstore'][4], storage_options={'anon': True})
CESM_hist_tas = xr.open_zarr(res.df['zstore'][5], storage_options={'anon': True})

# Store 245 scenario for CESM2
CESM_245_pr = xr.open_zarr(res.df['zstore'][7], storage_options={'anon': True})
CESM_245_tas = xr.open_zarr(res.df['zstore'][6], storage_options={'anon': True})

In [66]:
# Concatenate stored data of historical and 245 scenarios - for both models and atmospheric variables

# CanESM5 
CAN_pr = xr.concat([CAN_hist_pr, CAN_245_pr], dim = "time")
CAN_tas = xr.concat([CAN_hist_tas, CAN_245_tas], dim = "time")


# CESM2
CESM_tas = xr.concat([CESM_hist_pr, CESM_245_pr], dim = "time")
CESM_pr = xr.concat([CESM_hist_tas, CESM_245_tas], dim = "time")